# EWRI - Enhanced Wildfire Risk Index Pipeline
**Change COUNTY variable below to run for different counties**


In [24]:
# CELL 1: SETTINGS - CHANGE COUNTY HERE
COUNTY = "suffolk"  # Options: los_angeles, napa, suffolk, maricopa

CONFIGS = {
    "los_angeles": {"fire_name": "Bobcat Fire", "fire_year": 2020, "state_fips": "06", "county_fips": "037"},
    "napa": {"fire_name": "Glass Fire", "fire_year": 2020, "state_fips": "06", "county_fips": "055"},
    "suffolk": {"fire_name": "Pine Barrens Fire", "fire_year": 2020, "state_fips": "36", "county_fips": "103"},
    "maricopa": {"fire_name": "Bush Fire", "fire_year": 2020, "state_fips": "04", "county_fips": "013"}
}

config = CONFIGS[COUNTY]
FIRE_NAME, FIRE_YEAR = config["fire_name"], config["fire_year"]
STATE_FIPS, COUNTY_FIPS = config["state_fips"], config["county_fips"]

BASE = "/home/network-lab/Desktop/EWRI"
RAW = f"{BASE}/raw/{COUNTY}"
PROCESSED = f"{BASE}/processed/{COUNTY}"
OUTPUT = f"{BASE}/outputs/{COUNTY}"
H3_RES = 9

print(f"County: {COUNTY.upper()} | Fire: {FIRE_NAME} ({FIRE_YEAR})")


County: SUFFOLK | Fire: Pine Barrens Fire (2020)


In [13]:
# CELL 2: IMPORTS
import pandas as pd, geopandas as gpd, numpy as np, h3, rasterio, os, glob, warnings
from shapely.geometry import Point
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
warnings.filterwarnings('ignore')
os.makedirs(PROCESSED, exist_ok=True)
os.makedirs(OUTPUT, exist_ok=True)
print("Imports complete!")


Imports complete!


In [14]:
# CELL 3: HELPER FUNCTIONS
def geotiff_to_h3(filepath, res=9):
    with rasterio.open(filepath) as src:
        data, transform, nodata = src.read(1), src.transform, src.nodata
    mask = (data != nodata) if nodata else ~np.isnan(data)
    rows, cols = np.where(mask)
    results = []
    for r, c in zip(rows, cols):
        x, y = rasterio.transform.xy(transform, r, c)
        results.append({"h3_index": h3.latlng_to_cell(y, x, res), "value": float(data[r, c])})
    return pd.DataFrame(results).groupby("h3_index")["value"].mean().reset_index()

def h3_to_point(idx):
    lat, lng = h3.cell_to_latlng(idx)
    return Point(lng, lat)

print("Helper functions defined!")


Helper functions defined!


In [15]:
# CELL 4: CONVERT SATELLITE GeoTIFFs → H3
tifs = glob.glob(f"{RAW}/satellite_data/*.tif")
print(f"Found {len(tifs)} GeoTIFFs")

sat_dfs = []
for tif in tqdm(tifs, desc="Converting"):
    name = os.path.basename(tif).replace(".tif", "")
    df = geotiff_to_h3(tif, H3_RES)
    df = df.rename(columns={"value": name})
    sat_dfs.append(df)

sat_h3 = sat_dfs[0]
for df in sat_dfs[1:]:
    sat_h3 = sat_h3.merge(df, on="h3_index", how="outer")

sat_h3.to_parquet(f"{PROCESSED}/satellite_h3.parquet", index=False)
print(f"Satellite: {len(sat_h3):,} hexagons, {len(sat_h3.columns)-1} features")


Found 31 GeoTIFFs


Converting: 100%|██████████| 31/31 [56:17<00:00, 108.96s/it] 


Satellite: 301,779 hexagons, 31 features


In [ ]:
# CELL 5: CONVERT FEMA → H3 (SOVI only)
tracts = gpd.read_file(glob.glob(f"{RAW}/census_tracts/*.shp")[0])
tracts = tracts[(tracts["STATEFP"] == STATE_FIPS) & (tracts["COUNTYFP"] == COUNTY_FIPS)].to_crs("EPSG:4326")
print(f"Tracts: {len(tracts)}")

# Fix: Match filename containing "NRI_Table_CensusTracts", not just folder path
fema_csv = [f for f in glob.glob(f"{RAW}/fema_data/*/*.csv") if "NRI_Table_CensusTracts" in os.path.basename(f)][0]
print(f"FEMA file: {os.path.basename(fema_csv)}")
fema = pd.read_csv(fema_csv)[["TRACTFIPS", "SOVI_SCORE"]]  # SOVI only
fema["GEOID"] = fema["TRACTFIPS"].astype(str).str.zfill(11)

tracts_fema = tracts.merge(fema, on="GEOID", how="inner")
h3_data = []
for _, row in tqdm(tracts_fema.iterrows(), total=len(tracts_fema), desc="FEMA→H3"):
    try:
        for cell in h3.geo_to_cells(row.geometry.__geo_interface__, H3_RES):
            h3_data.append({"h3_index": cell, "SOVI_SCORE": row["SOVI_SCORE"]})
    except: pass

fema_h3 = pd.DataFrame(h3_data).groupby("h3_index").mean().reset_index()
fema_h3.to_parquet(f"{PROCESSED}/fema_h3.parquet", index=False)
print(f"FEMA (SOVI): {len(fema_h3):,} hexagons")


Tracts: 1009
FEMA file: NRI_Table_CensusTracts_Arizona.csv


FEMA→H3: 100%|██████████| 1009/1009 [00:01<00:00, 565.07it/s]


FEMA: 196,298 hexagons


In [25]:
# CELL 6: CONVERT VULNERABILITY DATA → H3 (Population + Buildings)
print("="*60)
print("CONVERTING VULNERABILITY DATA TO H3")
print("="*60)

from collections import defaultdict

vuln_dir = f"{RAW}/Vulnerability"
output_path = f"{PROCESSED}/exposure_h3.parquet"

# County name mapping for file names
county_names = {
    "los_angeles": "LA_County",
    "napa": "Napa_County", 
    "suffolk": "Suffolk_County",
    "maricopa": "Maricopa_County"
}
county_prefix = county_names[COUNTY]

# 1. POPULATION (WorldPop-style data for 3 years)
print("\n1. Processing population data...")
pop_data = defaultdict(lambda: {'pop_2017': 0, 'pop_2018': 0, 'pop_2019': 0})

for year in [2017, 2018, 2019]:
    tif_path = f"{vuln_dir}/{county_prefix}_Population_{year}.tif"
    print(f"   Loading: {os.path.basename(tif_path)}")
    
    with rasterio.open(tif_path) as src:
        data = src.read(1)
        transform = src.transform
        nodata = src.nodata
        
        # Find valid pixels (population > 0)
        if nodata is not None:
            mask = (data > 0) & (data != nodata)
        else:
            mask = (data > 0) & ~np.isnan(data)
        
        rows, cols = np.where(mask)
        for r, c in tqdm(zip(rows, cols), total=len(rows), desc=f"Pop {year}"):
            val = float(data[r, c])
            lng, lat = rasterio.transform.xy(transform, r, c)
            h3_idx = h3.latlng_to_cell(lat, lng, H3_RES)
            pop_data[h3_idx][f'pop_{year}'] += val

print(f"   Population hexagons: {len(pop_data):,}")

# 2. BUILDINGS (BuiltUp raster - sum values per hexagon)
print("\n2. Processing built-up data...")
building_data = defaultdict(float)

tif_path = f"{vuln_dir}/{county_prefix}_BuiltUp.tif"
print(f"   Loading: {os.path.basename(tif_path)}")

with rasterio.open(tif_path) as src:
    data = src.read(1)
    transform = src.transform
    nodata = src.nodata
    
    if nodata is not None:
        mask = (data > 0) & (data != nodata)
    else:
        mask = (data > 0) & ~np.isnan(data)
    
    rows, cols = np.where(mask)
    for r, c in tqdm(zip(rows, cols), total=len(rows), desc="BuiltUp"):
        val = float(data[r, c])
        lng, lat = rasterio.transform.xy(transform, r, c)
        h3_idx = h3.latlng_to_cell(lat, lng, H3_RES)
        building_data[h3_idx] += val

print(f"   Building hexagons: {len(building_data):,}")

# 3. COMBINE INTO ONE DATAFRAME
print("\n3. Combining population + buildings...")
all_h3 = set(pop_data.keys()) | set(building_data.keys())

records = []
for h3_idx in tqdm(all_h3, desc="Combining"):
    records.append({
        'h3_index': h3_idx,
        'pop_2017': pop_data[h3_idx]['pop_2017'],
        'pop_2018': pop_data[h3_idx]['pop_2018'],
        'pop_2019': pop_data[h3_idx]['pop_2019'],
        'built_up': building_data[h3_idx]
    })

exposure_df = pd.DataFrame(records)
exposure_df.to_parquet(output_path, index=False)

print(f"\n{'='*60}")
print(f"✓ Saved: {output_path}")
print(f"{'='*60}")
print(f"  Rows: {len(exposure_df):,}")
print(f"  Columns: {list(exposure_df.columns)}")


CONVERTING VULNERABILITY DATA TO H3

1. Processing population data...
   Loading: Suffolk_County_Population_2017.tif


Pop 2017: 100%|██████████| 281576/281576 [00:02<00:00, 99363.27it/s] 


   Loading: Suffolk_County_Population_2018.tif


Pop 2018: 100%|██████████| 281576/281576 [00:02<00:00, 100027.89it/s]


   Loading: Suffolk_County_Population_2019.tif


...

   Population hexagons: 22,457

2. Processing built-up data...
   Loading: Suffolk_County_BuiltUp.tif


BuiltUp: 100%|██████████| 5647015/5647015 [00:54<00:00, 103663.61it/s]


   Building hexagons: 58,569

3. Combining population + buildings...


Combining: 100%|██████████| 58572/58572 [00:00<00:00, 782771.03it/s]



✓ Saved: /home/network-lab/Desktop/EWRI/processed/suffolk/exposure_h3.parquet
  Rows: 58,572
  Columns: ['h3_index', 'pop_2017', 'pop_2018', 'pop_2019', 'built_up']
